In [2]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [3]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
selected_features = df[['koi_prad', 'koi_impact', 'koi_time0bk', 'koi_model_snr', 'koi_depth', 'koi_fpflag_ss', 'koi_srad', 'koi_insol', 'koi_slogg', 'koi_kepmag']]
selected_features.head()

,koi_prad,koi_impact,koi_time0bk,koi_model_snr,koi_depth,koi_fpflag_ss,koi_srad,koi_insol,koi_slogg,koi_kepmag
0,2.83,0.586,162.513840,25.8,874.8,0,0.927,9.11,4.467,15.347
1,14.60,0.969,175.850252,76.3,10829.0,1,0.868,39.30,4.544,15.436
2,33.46,1.276,170.307565,505.6,8079.2,1,0.791,891.96,4.564,15.597
3,2.75,0.701,171.595550,40.9,603.3,0,1.046,926.16,4.438,15.509
4,2.77,0.762,172.979370,40.2,686.0,0,0.972,427.65,4.486,15.714


In [9]:
from sklearn.preprocessing import LabelEncoder

y = df['koi_disposition']
X = selected_features

label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [20]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
# y_scaler = StandardScaler().fit(y_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transfor(y_train)
# y_test_scaled = y_scaler.transform(y_test)


In [21]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)

0.7505720823798627

In [24]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.16968102277977978, 'koi_model_snr'),
 (0.12565188701820093, 'koi_insol'),
 (0.12432992387728525, 'koi_prad'),
 (0.10167970067706353, 'koi_fpflag_ss'),
 (0.1010880770760016, 'koi_impact'),
 (0.09482867043952796, 'koi_depth'),
 (0.0947600991533632, 'koi_time0bk'),
 (0.06609952632304994, 'koi_kepmag'),
 (0.062174835268597385, 'koi_srad'),
 (0.05970625738713048, 'koi_slogg')]

In [26]:
import joblib
filename = 'model_saves/rf_exoplanet.sav'
joblib.dump(rf, filename)


['model_saves/rf_exoplanet.sav']